In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [5]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

ValueError: list.remove(x): x not in list

In [25]:
### 图像预处理
def preprocess(file_name):
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    chs = cv2.split(img)
    hue = ((chs[0] > 30) * 1) * chs[0]
    res = ((hue < 50) * 1) * hue

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    res = cv2.morphologyEx(res.astype(np.float32), cv2.MORPH_ERODE, kernel)
    resize = cv2.resize(res, (196, 196))
    resize = resize.reshape((1, 196, 196))
    resize = cv2.normalize(resize, resize, 0, 255, cv2.NORM_MINMAX)

    cv2.imwrite("test.jpg", resize[0])

    return resize

In [18]:
train_path = '/media/lor/data/data/plant-seedlings-classification/train'
train_samples = []
train_labels = []
label_to_idx = {}
for (i, cat) in enumerate(os.listdir(train_path)):
    label_to_idx[cat] = i
    cat_path = os.path.join(train_path, cat)
    print(cat_path)
    for item in os.listdir(cat_path):
        train_labels.append(i)
        train_samples.append(os.path.join(cat_path, item))
print(len(train_labels))
print(len(train_samples))

/media/lor/data/data/plant-seedlings-classification/train/Cleavers
/media/lor/data/data/plant-seedlings-classification/train/Maize
/media/lor/data/data/plant-seedlings-classification/train/Fat Hen
/media/lor/data/data/plant-seedlings-classification/train/Sugar beet
/media/lor/data/data/plant-seedlings-classification/train/Charlock
/media/lor/data/data/plant-seedlings-classification/train/Loose Silky-bent
/media/lor/data/data/plant-seedlings-classification/train/Small-flowered Cranesbill
/media/lor/data/data/plant-seedlings-classification/train/Common Chickweed
/media/lor/data/data/plant-seedlings-classification/train/Shepherds Purse
/media/lor/data/data/plant-seedlings-classification/train/Black-grass
/media/lor/data/data/plant-seedlings-classification/train/Scentless Mayweed
/media/lor/data/data/plant-seedlings-classification/train/Common wheat
4750
4750


In [27]:
class PlantDS(torch.utils.data.Dataset):
    def __init__(self, samples, targets):
        self.samples = samples
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        target = self.targets[idx]
        sample = preprocess(self.samples[idx])
        return (target, sample)

In [28]:
train_ds = PlantDS(train_samples, train_labels)

In [30]:
train_dloader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4)

In [31]:
class PlantNet(nn.Module):
    def __init__(self):
        super(PlantNet, self).__init__()
        